## Example: Preparing a data catalog

This example illustrates the how to prepare your own HydroMT [DataCatalog](https://deltares.github.io/hydromt/latest/_generated/hydromt.data_catalog.DataCatalog.html) to reference your own data sources and start using then within HydroMT, see [user guide](https://deltares.github.io/hydromt/latest/user_guide/data_prepare_cat.html).

In [ ]:
# import python libraries
import os
from pprint import pprint
import matplotlib.pyplot as plt
import xarray as xr
import rioxarray
import geopandas as gpd
import pandas as pd

# import hydromt and setup logging
import hydromt
from hydromt.log import setuplog

logger = setuplog("prepare data catalog", log_level=10)

The steps to use your own data within HydroMT are in brief:

  1) **Have your (local) dataset ready** in one of the supported [raster](https://deltares.github.io/hydromt/latest/user_guide/data_types.html#raster-formats) (tif, ascii, netcdf, zarr...), 
   [vector](https://deltares.github.io/hydromt/latest/user_guide/data_types.html#vector-formats) (shp, geojson, gpkg...) or [geospatial time-series](https://deltares.github.io/hydromt/latest/user_guide/data_types.html#geo-formats) (netcdf, csv...) format.
  2) **Create your own [yaml file](https://deltares.github.io/hydromt/latest/user_guide/data_prepare_cat.html#data-yaml)** with a reference to your prepared dataset and properties (path, data_type, driver, etc.) following the HydroMT [data conventions](https://deltares.github.io/hydromt/latest/user_guide/data_conventions.html#data-convention). For this step, you can also start from an existing pre-defined catalog or use it for inspiration.

The existing pre-defined catalog are:

In [ ]:
# this download the artifact_data archive v0.0.6
data_catalog = hydromt.DataCatalog(data_libs=["artifact_data=v0.0.8"])
pprint(data_catalog.predefined_catalogs)

In this notebook, we will see how we can create a data catalog for several type of input data. For this we have prepared several type of data that we will catalogue, let's see which data we have available:

In [ ]:
# the artifact data is stored in the following location
root = os.path.join(data_catalog._cache_dir, "artifact_data", "v0.0.8")
# let's print some of the file that are there
for item in os.listdir(root)[-10:]:
    print(item)

### RasterDataset from raster file

The first file we will use is a 'simple' raster file in a tif format: **vito.tif**. This file contains a landuse classification raster. The first thing to do before adding a new file to a data catalog is to get to know what is inside of our file mainly:

  - **location of the file**: `path`.
  - **type of data**: `data_type`. `RasterDataset` for gridded data, `GeoDataFrame` for vector data, `GeoDataset` for point timeseries and `DataFrame` for tabular data.
  - **file format**: `driver`. The file format impacts the driver or python function that will be used to open the data. Either `raster`, `raster_tindex`, `netcdf`, `zarr`, `vector`, `vector_table`.
  - **crs**: `crs`. Coordinate sytem of the data. Optional as it is usually encoded in the data itself.
  - **variables and their properties**: `rename`, `unit_mult`, `unit_add`. Looking at the variables in the input data and what are their names and units so that we can convert them to the [HydroMT data conventions](https://deltares.github.io/hydromt/latest/user_guide/data_conventions.html).
  
There are more arguments or properties to look for that are explained in more detailed in the [documentation](https://deltares.github.io/hydromt/latest/user_guide/data_prepare_cat.html). To discover our data we can either use GIS software like QGIS or GDAL or just use python directly to try and open the data.

Let's open our vito.tif file with xarray and rioxarray:

In [ ]:
da = xr.open_dataarray(os.path.join(root, "vito.tif"))
pprint(da)
print(f"CRS: {da.raster.crs}")
da.plot()

What we see is that we have a simple raster with landuse data in crs 4326. Let's translate what we know into a data catalog.

In [ ]:
yml_str = f"""
meta:
  root: {root}
  
vito:
    path: vito.tif
    data_type: RasterDataset
    driver: raster
    crs: 4326
    meta:
        category: landuse
"""
data_lib = "tmpdir/vito.yml"
with open(data_lib, mode="w") as f:
    f.write(yml_str)

And let's now see if HydroMT can properly read the file from the data catalog we prepared:

In [ ]:
data_catalog = hydromt.DataCatalog(data_libs=[data_lib], logger=logger)
da = data_catalog.get_rasterdataset("vito")
da

### RasterDataset from several raster files

The second file we will add is the **merit_hydro** which consists of elevation and elevation-derived variables stored in several tif files for each variable.  Let's see what are their names.

In [ ]:
folder_name = os.path.join(root, "merit_hydro")
# let's see which files are there
for path, _, files in os.walk(folder_name):
    print(path)
    for name in files:
        print(f" - {name}")

We have here 9 files. When reading tif files, the name of the file is used as the variable name. HydroMT uses data conventions to ensure that certain variables should have the same name and units to be used in automatically in the workflows. For example elevation data should be called *elevtn* with unit in [m asl]. Check the [data conventions](https://deltares.github.io/hydromt/latest/user_guide/data_conventions.html) and see if you need to ``rename`` or change units with ``unit_add`` and ``unit_mult`` for this dataset in the data catalog. 

Here all names and units are correct, so we just show an example were we rename the *hnd* variable.

In [ ]:
yml_str = f"""
meta:
  root: {root}

merit_hydro:
  data_type: RasterDataset
  driver: raster
  driver_kwargs:
    chunks:
      x: 6000
      y: 6000
  meta:
    category: topography
  rename:
    hnd: height_above_nearest_drain
  path: merit_hydro/*.tif
"""
# overwrite data catalog
data_lib = "tmpdir/merit_hydro.yml"
with open(data_lib, mode="w") as f:
    f.write(yml_str)

In [ ]:
data_catalog.from_yml(data_lib)  # add a yaml file to the data catalog
print(data_catalog.sources.keys())
ds = data_catalog.get_rasterdataset("merit_hydro")
ds

In the ``path``, the filenames can be further specified with *{variable}*, *{year}* and *{month}* keys to limit which files are being read based on the get_data request in the form of *"path/to/my/files/{variable}_{year}_{month}.nc"*. 

Let's see how this works:

In [ ]:
# NOTE: the double curly brackets will be printed as single brackets in the text file
yml_str = f"""
meta:
  root: {root}

merit_hydro:
  data_type: RasterDataset
  driver: raster
  driver_kwargs:
    chunks:
      x: 6000
      y: 6000
  meta:
    category: topography
  rename:
    hnd: height_above_nearest_drain
  path: merit_hydro/{{variable}}.tif
"""
# overwrite data catalog
data_lib = "tmpdir/merit_hydro.yml"
with open(data_lib, mode="w") as f:
    f.write(yml_str)

In [ ]:
data_catalog.from_yml(data_lib)  # add a yaml file to the data catalog
print(data_catalog.sources.keys())
ds = data_catalog.get_rasterdataset(
    "merit_hydro", variables=["height_above_nearest_drain", "elevtn"]
)
ds

### RasterDataset from a netcdf file

The last RasterDataset file we will add is the **era5.nc** which consists of climate variables stored in a netcdf file.  Let's open this file with xarray.

In [ ]:
ds = xr.open_dataset(os.path.join(root, "era5.nc"))
pprint(ds)

In [ ]:
# Select first timestep
ds1 = ds.sel(time=ds.time[0])
ds1
# Plot
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 3))
ds1["precip"].plot(ax=axes[0])
axes[0].set_title("precip")
ds1["temp"].plot(ax=axes[1])
axes[1].set_title("temp")
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 3))
ds1["kin"].plot(ax=axes[0])
axes[0].set_title("kin")
ds1["press_msl"].plot(ax=axes[1])
axes[1].set_title("press_msl")

Checking the [data conventions](https://deltares.github.io/hydromt/latest/user_guide/data_conventions.html) we see that all variables already have the right names but the units should be changed:

  - precip from m to mm
  - temp, temp_min, temp_max from K to C
  - kin, kout from J.m-2 to W.m-2
  - press_msl from Pa to hPa

Let's change the units using ``unit_mult`` and ``unit_add``:

In [ ]:
yml_str = f"""
meta:
  root: {root}

era5:
  crs: 4326
  data_type: RasterDataset
  driver: netcdf
  meta:
    category: meteo
    history: Extracted from ERA5 daily data on pressure levels
  path: era5.nc
  unit_add:
    temp: -273.15
    temp_max: -273.15
    temp_min: -273.15
    time: 86400
  unit_mult:
    kin: 0.000277778
    kout: 0.000277778
    precip: 1000
    press_msl: 0.01

"""
# overwrite data catalog
data_lib = "tmpdir/era5.yml"
with open(data_lib, mode="w") as f:
    f.write(yml_str)

And now open our dataset and check the units have been converted.

In [ ]:
data_catalog.from_yml(data_lib)  # add a yaml file to the data catalog
print(data_catalog.sources.keys())
ds = data_catalog.get_rasterdataset("era5")
ds

In [ ]:
# Select first timestep
ds1 = ds.sel(time=ds.time[0])
ds1
# Plot
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 3))
ds1["precip"].plot(ax=axes[0])
axes[0].set_title("precip")
ds1["temp"].plot(ax=axes[1])
axes[1].set_title("temp")
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 3))
ds1["kin"].plot(ax=axes[0])
axes[0].set_title("kin")
ds1["press_msl"].plot(ax=axes[1])
axes[1].set_title("press_msl")

### GeoDataFrame from a vector file

Now we will see how to add vector data to the data catalogue based on **rivers_lin2019_v1.gpkg**. Vector files can be open in Python with geopandas (or you can use QGIS) to inspect the data.

In [ ]:
gdf = gpd.read_file(os.path.join(root, "rivers_lin2019_v1.gpkg"))
pprint(gdf.head())
print(f"Variables: {gdf.columns}")
print(f"CRS: {gdf.crs}")
gdf.plot()

This data source contains rivers line including attributes that can be usefull to setup models such as river width, average discharge or bankfull discharge. Here it's not needed but feel free to try out some renaming or unit conversion. The minimal data catalog input would be:

In [ ]:
yml_str = f"""
meta:
  root: {root}

rivers_lin:
  data_type: GeoDataFrame
  driver: vector
  meta:
    category: hydrography
  path: rivers_lin2019_v1.gpkg
"""
# overwrite data catalog
data_lib = "tmpdir/rivers.yml"
with open(data_lib, mode="w") as f:
    f.write(yml_str)

In [ ]:
data_catalog.from_yml(data_lib)  # add a yaml file to the data catalog
print(data_catalog.sources.keys())
gdf = data_catalog.get_geodataframe("rivers_lin")

### GeoDataset from a netcdf file

Now we will see how to add geodataset data to the data catalogue based on **gtsmv3_eu_era5.nc**. This geodataset file contains ocean water level timeseries at specific stations locations in netdcf format and can be opened in Python with xarray. In HydroMT we use a specific wrapper around xarray called GeoDataset to mark that this file contains geospatial timeseries, in this case point timeseries. But for now we can inspect it with xarray.

To learn more about GeoDataset type you can check the [reading geodataset example](https://deltares.github.io/hydromt/latest/_examples/reading_point_data.html).

In [ ]:
ds = xr.open_dataset(os.path.join(root, "gtsmv3_eu_era5.nc"))
ds

This is quite a classic file, so the data catalog entry is quite straightforward:

In [ ]:
yml_str = f"""
meta:
  root: {root}

gtsm:
  crs: 4326
  data_type: GeoDataset
  driver: netcdf
  meta:
    category: ocean
    source_version: GTSM v3.0
  path: gtsmv3_eu_era5.nc
"""
# overwrite data catalog
data_lib = "tmpdir/gtsm.yml"
with open(data_lib, mode="w") as f:
    f.write(yml_str)

In [ ]:
data_catalog.from_yml(data_lib)  # add a yaml file to the data catalog
print(data_catalog.sources.keys())
ds = data_catalog.get_geodataset("gtsm")
ds

### GeoDataset from vector files

For geodataset, you can also use the ``vector`` driver to combine two files, one for the location and one for the timeseries into one geodataset. We have a custom example available in the data folder of our example notebook using the files *stations.csv* for the locations and *stations_data.csv* for the timeseries:

In [ ]:
# example data folder
root_data = "data"
# let's print some of the file that are there
for item in os.listdir(root_data):
    print(item)

For this driver to work, the format of the timeseries table is quite strict (see [docs](https://deltares.github.io/hydromt/latest/user_guide/data_types.html#csv-point-time-series-data)). Let's inspect the two files using pandas in python:

In [ ]:
df_locs = pd.read_csv("data/stations.csv")
pprint(df_locs)

In [ ]:
df_data = pd.read_csv("data/stations_data.csv")
pprint(df_data)

And see how the data catalog would look like:

In [ ]:
yml_str = f"""
meta:
  root: {os.path.join(os.getcwd(), 'data')}

waterlevel_csv:
  crs: 4326
  data_type: GeoDataset
  driver: vector
  driver_kwargs:
    fn_data: stations_data.csv
  path: stations.csv
  rename:
    stations_data: waterlevel

"""
# overwrite data catalog
data_lib = "tmpdir/waterlevel.yml"
with open(data_lib, mode="w") as f:
    f.write(yml_str)

In [ ]:
data_catalog.from_yml(data_lib)  # add a yaml file to the data catalog
print(data_catalog.sources.keys())
ds = data_catalog.get_geodataset("waterlevel_csv")
ds